<a href="https://colab.research.google.com/github/aloyindependent404/npc_game/blob/main/Chatbot_Pembaca_PDF_Kontrak.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q langchain langchain-openai langchain-community langchain-core faiss-cpu pypdf tiktoken sentence-transformers pandas


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.0/69.0 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.9/438.9 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.6/304.6 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 56.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 53.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/

🔹 1. Chatbot Pembaca PDF Kontrak

📚 Fungsi: Upload dokumen PDF → Agent bisa menjawab pertanyaan berdasarkan isi kontrak.
Level: RAG dasar (Retriever + Prompt + LLM)
Teknologi: LangChain, PyPDFLoader, FAISS, SentenceTransformer

🧩 Cocok untuk: kontrak kerja, FAQ, atau skrips

In [23]:
import os
import pandas as pd
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.tools import Tool
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import SentenceTransformerEmbeddings
from langchain.chains import RetrievalQA

from getpass import getpass
os.environ["OPENAI_API_KEY"] = getpass("masukkan api kamu")

llm = ChatOpenAI(
    model="Qwen/Qwen3-235B-A22B-fp8-tput",
    base_url="https://api.together.xyz/v1",
    api_key=os.environ["OPENAI_API_KEY"],
    temperature=0.2,
    max_tokens=512
)

result = f"api berhasil di pasang {llm}"
print(result)

masukkan api kamu··········
api berhasil di pasang client=<openai.resources.chat.completions.completions.Completions object at 0x78959a1870d0> async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x78959a185e50> root_client=<openai.OpenAI object at 0x7895aa67ce50> root_async_client=<openai.AsyncOpenAI object at 0x78959a18e890> model_name='Qwen/Qwen3-235B-A22B-fp8-tput' temperature=0.2 model_kwargs={} openai_api_key=SecretStr('**********') openai_api_base='https://api.together.xyz/v1' max_tokens=512


In [18]:

rag_prompt = PromptTemplate.from_template("""
Kamu adalah asisten hukum cerdas dan jenaka. Tugasmu adalah membaca dokumen kontrak dan menjawab pertanyaan berdasarkan isinya, dengan gaya yang kritis dan lucu seperti stand-up comedy. Jangan mengarang, cukup berdasarkan dokumen.

### KONTEKS DOKUMEN:
{context}

### PERTANYAAN:
{question}

### PETUNJUK:
- Jawaban harus lucu, kritis, dan bernada stand-up comedy.
- Jika jawabannya tidak ada di dokumen, katakan: "Informasi tersebut tidak tersedia dalam kontrak."
- Jangan tambahkan informasi dari luar dokumen.
- Jangan tampilkan reasoning dalam tag <think>
""")

chain = rag_prompt | llm

In [21]:
loaders = PyPDFLoader("sample_data/kontrak.pdf")
docs = loaders.load_and_split()
embending = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
vectore_store = FAISS.from_documents(docs, embending)
reetriver = vectore_store.as_retriever(search_kwargs={"k": 3})


# Bangun RAG Chain
rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=reetriver,
    chain_type_kwargs={"prompt": rag_prompt}
)

# Contoh pemakaian
pertanyaan = "benefit pekerjaan ini apa aja"
jawaban = rag_chain.run(pertanyaan)
print(jawaban)


<think>

</think>

Oh, jadi kita mau bahas *benefit* kerjaan ini, ya? Mari kita lihat di kontrak... *eh, ternyata ini bukan perusahaan startup yang suka kasih fasilitas kekinian kayak kopi sepuasnya atau meja pijat*. Nih, *benefit*-nya:

1. **Gaji** – Iya, gaji ada, tapi jumlahnya malah dikosongin di kontrak. Seperti janji mantan: *“Nanti aku kasih tahu besok”* dan besoknya ilang sampai hari kiamat. Gajinya juga dipotong pajak, jadi dapatnya ya sisa yang sudah "dimangkas negara".

2. **Dipaksa Lembur** – Kalau ada kerjaan mendesak, kamu *wajib* lembur. Ini *benefit*, lho! Siapa yang nggak mau kerja lebih, tanpa upah lembur yang diatur di kontrak? *Pure passion*, bro!

3. **Izin Bisa, Tapi Harus Mikir Dua Kali** – Mau izin? Boleh, tapi harus dua hari sebelumnya. Kalau ada hal mendadak seperti sakit atau kerabat meninggal? *Tenaaang*, masih bisa ajukan izin mendadak. Tapi tetap, *nggak ada jaminan libur*. Ini bukan cuti, ini *kompensasi*, kayak kasih selamat ulang tahun ke temen yang lup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')